In [1]:
import sys
print(sys.path)
import pandas as pd
import json
from tqdm import tqdm
import csv
import ast
from sklearn import preprocessing
import numpy as np
from math import ceil
import torch
import pickle
import joblib
import re
from sklearn.preprocessing import MultiLabelBinarizer
import json
from ICDMappings import ICDMappings # Convert from ICD9 to CCS
import gc
import datetime

isa = '/Users/isabella/Documents/IntelligentCare/DICE/DICE-new/' # Put your own oath

['/Users/isabella/Documents/IntelligentCare/DICE/DICE-new/preprocessing', '/Users/isabella/.vscode/extensions/ms-toolsai.jupyter-2022.9.1303220346/pythonFiles', '/Users/isabella/.vscode/extensions/ms-toolsai.jupyter-2022.9.1303220346/pythonFiles/lib/python', '/opt/anaconda3/envs/DICE_py38/lib/python38.zip', '/opt/anaconda3/envs/DICE_py38/lib/python3.8', '/opt/anaconda3/envs/DICE_py38/lib/python3.8/lib-dynload', '', '/opt/anaconda3/envs/DICE_py38/lib/python3.8/site-packages']


/opt/anaconda3/envs/DICE_py38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Initialization ICD9 mapping to CCS
icdmap = ICDMappings()

# Check available groupers
icdmap.get_available_groupers()

['icd9toccs',
 'icd9_3toccs',
 'ccstodescription',
 'icd9to10',
 'icd10to9',
 'icd9tochapter',
 'icd9_level3',
 'icd9tocci',
 'icd9checker']

### Load time window data

In [3]:
# Read time window data
"""
# assert 1 == 2, 'Preventing myself to run this cell by accident'
file = isa + '../icare-dataset_2022-11-alunos/students_obs_6_pred_3_overlap_0_int_urg.csv'
i = 0 # limiter (to test a couple of chunks)
# keep preprocessed chunks (aka: dataframes)
final_result = list()

with pd.read_csv(file,
                 sep='$',
                 chunksize=1_000_000,
                 #usecols=[0,2,3,4,5,7,8,13], # choose columns right off the bat when reading. I believe it speeds up stuff
                 dtype={'NHC':str,
                        'DATA_ADMISSAO':str,
                        'VECTOR':object,
                        'INFO':object,
                        'observation_events':str,
                        'info_observation_events':object, 
                        'prediction_events':str, 
                        'prediction_time':str
                       },
                 # parse_dates=['DATA_ADMISSAO'], 
                 # date_parser= lambda x: pd.to_datetime(x,format='%Y-%m-%d %H:%M:%S')
                ) as reader:
    for idx,chunk in tqdm(enumerate(reader)):
        # process dataframe (eg: change dtypes to reduce size, choose columns, preprocess stuff)
        df_chunk = chunk[chunk["NHC"].isin(diabetes_patients)]
        final_result.append(df_chunk)
# concat everything at the end. way faster than concating at each step (learned the hard way)

final_result = pd.concat(final_result)
"""

'\n# assert 1 == 2, \'Preventing myself to run this cell by accident\'\nfile = isa + \'../icare-dataset_2022-11-alunos/students_obs_6_pred_3_overlap_0_int_urg.csv\'\ni = 0 # limiter (to test a couple of chunks)\n# keep preprocessed chunks (aka: dataframes)\nfinal_result = list()\n\nwith pd.read_csv(file,\n                 sep=\'$\',\n                 chunksize=1_000_000,\n                 #usecols=[0,2,3,4,5,7,8,13], # choose columns right off the bat when reading. I believe it speeds up stuff\n                 dtype={\'NHC\':str,\n                        \'DATA_ADMISSAO\':str,\n                        \'VECTOR\':object,\n                        \'INFO\':object,\n                        \'observation_events\':str,\n                        \'info_observation_events\':object, \n                        \'prediction_events\':str, \n                        \'prediction_time\':str\n                       },\n                 # parse_dates=[\'DATA_ADMISSAO\'], \n                 # date_parser

In [4]:
# final_result

In [5]:
# final_result.to_csv('final2.csv')

In [6]:
# Load time-window dataset: observation period of 6 months, prediction horizon of 3 months (without overlap)
# Outcome: hospitalization and urgency visits
table = pd.read_csv(isa + 'final2.csv',  index_col=[0])
print(len(table))
table.head()

250418


,NHC,DATA_ADMISSAO,VECTOR,INFO,observation_events,info_observation_events,prediction_events,prediction_time
151,01412FD488D32C591826DEC62C02B178,"[2359-07-04, 2360-02-13, 2359-07-11, 2361-07-0...","[[2359-07-04, Ambulatório, 85.0, Medicina Inte...","[[2359-07-04, DEMO, DEMO, DEMO, DEMO, ICD9, CR...",[],[],False,2340-07-08
618,050E246D80AA2F1BACD35F4C06CB1525,"[2358-01-19, 2353-01-02, 2355-10-25, 2358-10-2...","[[2358-01-19, Ambulatório, 72.0, Urologia, M],...","[[2358-01-19, DEMO, DEMO, DEMO, DEMO], [2353-0...",[],[],False,2340-07-08
641,054760511AEF49C14D9F2A98A70C98C5,"[2356-06-28, 2356-08-05, 2356-07-16, 2360-07-0...","[[2356-06-28, Ambulatório, 59.0, Radiologia, M...","[[2356-06-28, DEMO, DEMO, DEMO, DEMO], [2356-0...",[],[],False,2340-07-08
1050,08981B30B3CE45D82CE414AF73A3F48F,"[2353-03-15, 2358-03-16, 2351-07-01, 2350-07-3...","[[2353-03-15, Ambulatório, 68.0, Dermatologia,...","[[2353-03-15, DEMO, DEMO, DEMO, DEMO], [2358-0...",[],[],False,2340-07-08
1481,0C564E336B0DCB433EA6C9D072456383,"[2361-08-25, 2360-10-17, 2360-11-17, 2361-08-2...","[[2361-08-25, Ambulatório, 50.0, Radiologia, F...","[[2361-08-25, DEMO, DEMO, DEMO, DEMO], [2360-1...",[],[],False,2340-07-08


In [7]:
table = table.reset_index(drop=True)
table.head()

,NHC,DATA_ADMISSAO,VECTOR,INFO,observation_events,info_observation_events,prediction_events,prediction_time
0,01412FD488D32C591826DEC62C02B178,"[2359-07-04, 2360-02-13, 2359-07-11, 2361-07-0...","[[2359-07-04, Ambulatório, 85.0, Medicina Inte...","[[2359-07-04, DEMO, DEMO, DEMO, DEMO, ICD9, CR...",[],[],False,2340-07-08
1,050E246D80AA2F1BACD35F4C06CB1525,"[2358-01-19, 2353-01-02, 2355-10-25, 2358-10-2...","[[2358-01-19, Ambulatório, 72.0, Urologia, M],...","[[2358-01-19, DEMO, DEMO, DEMO, DEMO], [2353-0...",[],[],False,2340-07-08
2,054760511AEF49C14D9F2A98A70C98C5,"[2356-06-28, 2356-08-05, 2356-07-16, 2360-07-0...","[[2356-06-28, Ambulatório, 59.0, Radiologia, M...","[[2356-06-28, DEMO, DEMO, DEMO, DEMO], [2356-0...",[],[],False,2340-07-08
3,08981B30B3CE45D82CE414AF73A3F48F,"[2353-03-15, 2358-03-16, 2351-07-01, 2350-07-3...","[[2353-03-15, Ambulatório, 68.0, Dermatologia,...","[[2353-03-15, DEMO, DEMO, DEMO, DEMO], [2358-0...",[],[],False,2340-07-08
4,0C564E336B0DCB433EA6C9D072456383,"[2361-08-25, 2360-10-17, 2360-11-17, 2361-08-2...","[[2361-08-25, Ambulatório, 50.0, Radiologia, F...","[[2361-08-25, DEMO, DEMO, DEMO, DEMO], [2360-1...",[],[],False,2340-07-08


In [8]:
table = table.dropna(how='any',axis=0) 
print(len(table))
table.head()

250418


,NHC,DATA_ADMISSAO,VECTOR,INFO,observation_events,info_observation_events,prediction_events,prediction_time
0,01412FD488D32C591826DEC62C02B178,"[2359-07-04, 2360-02-13, 2359-07-11, 2361-07-0...","[[2359-07-04, Ambulatório, 85.0, Medicina Inte...","[[2359-07-04, DEMO, DEMO, DEMO, DEMO, ICD9, CR...",[],[],False,2340-07-08
1,050E246D80AA2F1BACD35F4C06CB1525,"[2358-01-19, 2353-01-02, 2355-10-25, 2358-10-2...","[[2358-01-19, Ambulatório, 72.0, Urologia, M],...","[[2358-01-19, DEMO, DEMO, DEMO, DEMO], [2353-0...",[],[],False,2340-07-08
2,054760511AEF49C14D9F2A98A70C98C5,"[2356-06-28, 2356-08-05, 2356-07-16, 2360-07-0...","[[2356-06-28, Ambulatório, 59.0, Radiologia, M...","[[2356-06-28, DEMO, DEMO, DEMO, DEMO], [2356-0...",[],[],False,2340-07-08
3,08981B30B3CE45D82CE414AF73A3F48F,"[2353-03-15, 2358-03-16, 2351-07-01, 2350-07-3...","[[2353-03-15, Ambulatório, 68.0, Dermatologia,...","[[2353-03-15, DEMO, DEMO, DEMO, DEMO], [2358-0...",[],[],False,2340-07-08
4,0C564E336B0DCB433EA6C9D072456383,"[2361-08-25, 2360-10-17, 2360-11-17, 2361-08-2...","[[2361-08-25, Ambulatório, 50.0, Radiologia, F...","[[2361-08-25, DEMO, DEMO, DEMO, DEMO], [2360-1...",[],[],False,2340-07-08


In [9]:
len(table.NHC.unique())

4039

In [10]:
table.NHC.value_counts()

01412FD488D32C591826DEC62C02B178    62
FD915917AF073119CCD27097FCEF4A17    62
E6F11FC6386F547EADA7F855AB74A2CD    62
E7107B9C839B51050DB4DA11428A3DCF    62
E8E67AF85F856DBF9B453DB03968D3B9    62
                                    ..
A825B03188C75CD9403462C865130D9E    62
AA2C632FB376B5288D0325594D3E515A    62
AA9D4F822E617D7AE40D3AE0A83668D1    62
ABE111A9E437C998FDB27FACDBCB4724    62
FEB96E6637ACCF6B46A683F4C68A5EC5    62
Name: NHC, Length: 4039, dtype: int64

In [11]:
def nth_repl(s, sub, repl, n):
    """
    Replace nth element of string.
    """
    find = s.find(sub)
    # If find is not -1 we have found at least one match for the substring
    i = find != -1
    # Loop util we find the nth or we find no match
    while find != -1 and i != n:
        # Find + 1 means we start searching from after the last match
        find = s.find(sub, find + 1)
        i += 1
    # If i is equal to n we found nth match so replace
    if i == n:
        return s[:find] + repl + s[find+len(sub):]
    return s

In [12]:
def replace_string(s):
    """
    Replaces string to use eval method (turns into vector format).
    """
    
    # Replace [ for ["
    substr = "["
    inserttxt = "\""
    s = s.replace(substr, substr+inserttxt)
    
    # Replace ] for "]
    substr = "]"
    inserttxt = "\""
    s = s.replace(substr, inserttxt+substr)
    
    # Replace first appearance of [" for [
    s = nth_repl(s, "[\"", "[", 1)
    
    # Replace last appearance of "] for ]
    substr = "\"]"
    inserttxt = "]"
    pos= s.rfind(substr)
    if pos > -1:
        # Replace last occurrences of substring 
        s = s[:pos] + inserttxt + s[pos + len(substr):]
        
    return s

In [13]:
def order_date(table_vector):
    """
    Order events by date.
    """
    aux = eval(replace_string(table_vector))
    return sorted(aux, key=lambda x: x[0])    

In [14]:
order_date(table.INFO[0])

[['2347-09-28, DEMO, DEMO, DEMO, DEMO'],
 ['2348-09-03, DEMO, DEMO, DEMO, DEMO, DEMO'],
 ['2349-03-17, DEMO, DEMO, DEMO, DEMO'],
 ['2350-07-14, DEMO, DEMO, DEMO, DEMO'],
 ['2351-08-24, DEMO, DEMO, DEMO, DEMO'],
 ['2351-10-26, DEMO, DEMO, DEMO, DEMO, ATC, ATC'],
 ['2351-12-14, DEMO, DEMO, DEMO, DEMO'],
 ['2351-12-14, DEMO, DEMO, DEMO, DEMO, ATC, ATC'],
 ['2351-12-16, DEMO, DEMO, DEMO, DEMO, ATC'],
 ['2352-04-13, DEMO, DEMO, DEMO, DEMO'],
 ['2352-04-24, DEMO, DEMO, DEMO, DEMO'],
 ['2352-06-01, DEMO, DEMO, DEMO, DEMO'],
 ['2352-06-30, DEMO, DEMO, DEMO, DEMO'],
 ['2353-04-02, DEMO, DEMO, DEMO, DEMO'],
 ['2353-04-07, DEMO, DEMO, DEMO, DEMO'],
 ['2353-04-17, DEMO, DEMO, DEMO, DEMO'],
 ['2353-05-01, DEMO, DEMO, DEMO, DEMO'],
 ['2353-09-27, DEMO, DEMO, DEMO, DEMO'],
 ['2353-09-27, DEMO, DEMO, DEMO, DEMO'],
 ['2357-05-13, DEMO, DEMO, DEMO, DEMO'],
 ['2357-05-19, DEMO, DEMO, DEMO, DEMO, DEMO, ATC, ATC, ATC, ATC, ATC, ATC, ATC, ATC, ATC, ATC, ATC, ATC, ATC, ATC, ATC, ATC'],
 ['2357-05-22, DEMO, D

In [15]:
# Order data by date
table['VECTOR'] = table['VECTOR'].map(lambda s: order_date(s))

In [16]:
# Order data by date
table['INFO'] = table['INFO'].map(lambda s: order_date(s))

In [17]:
table['VECTOR'][250417]

[['2348-02-06, Ambulatório, 48.0, Ortopedia, M'],
 ['2348-02-07, Ambulatório, 48.0, Radiologia, M'],
 ['2348-02-21, Ambulatório, 48.0, Dermatologia, M'],
 ['2348-02-21, Ambulatório, 48.0, Patologia Clínica, M, 4544-3, 2951-2, 718-7, 789-8, 777-3, 785-6, 1742-6, 11572-5, 1920-8, 6690-2, 787-2, 788-0, 786-4, 4537-7, 713-8, 14933-6, 5905-5, 13929-5, 736-9, 22664-7, 2324-2, 2823-3, 770-8, 14682-9, 706-2'],
 ['2348-03-12, Ambulatório, 48.0, Ortopedia, M'],
 ['2349-04-03, Urgência, 49.0, AMP, Radiologia, M, N02BE01, A03FA01'],
 ['2349-04-30, Ambulatório, 49.0, Cardiologia, M'],
 ['2349-05-22, Ambulatório, 50.0, Radiologia, M'],
 ['2353-03-05, Ambulatório, 53.0, Radiologia, M'],
 ['2355-12-30, Ambulatório, 56.0, Oftalmologia, M'],
 ['2356-08-05, Ambulatório, 57.0, Angiologia e Cirurgia Vascular, M'],
 ['2356-11-17, Ambulatório, 57.0, Medicina Interna, M'],
 ['2358-04-27, Ambulatório, 58.0, Angiologia e Cirurgia Vascular, M'],
 ['2358-05-01, Ambulatório, 58.0, Medicina Interna, M'],
 ['2358-05

In [18]:
table['VECTOR'][250417][30][0].split(", ")

['2361-06-12', 'Ambulatório', '62.0', 'Radiologia', 'M']

In [19]:
# Transform each item of list into string and remove dates, important for future manipulations (takes around 5min)
for item in range(len(table.VECTOR)):
    for it in range(len(table.VECTOR[item])):
        table['VECTOR'][item][it] = table['VECTOR'][item][it][0].split(", ")[1:]
        table['INFO'][item][it] = table['INFO'][item][it][0].split(", ")[1:]

In [20]:
table['INFO'][250417]

[['DEMO', 'DEMO', 'DEMO', 'DEMO'],
 ['DEMO', 'DEMO', 'DEMO', 'DEMO'],
 ['DEMO', 'DEMO', 'DEMO', 'DEMO'],
 ['DEMO',
  'DEMO',
  'DEMO',
  'DEMO',
  'LABS',
  'LABS',
  'LABS',
  'LABS',
  'LABS',
  'LABS',
  'LABS',
  'LABS',
  'LABS',
  'LABS',
  'LABS',
  'LABS',
  'LABS',
  'LABS',
  'LABS',
  'LABS',
  'LABS',
  'LABS',
  'LABS',
  'LABS',
  'LABS',
  'LABS',
  'LABS',
  'LABS',
  'LABS'],
 ['DEMO', 'DEMO', 'DEMO', 'DEMO'],
 ['DEMO', 'DEMO', 'DEMO', 'DEMO', 'DEMO', 'ATC', 'ATC'],
 ['DEMO', 'DEMO', 'DEMO', 'DEMO'],
 ['DEMO', 'DEMO', 'DEMO', 'DEMO'],
 ['DEMO', 'DEMO', 'DEMO', 'DEMO'],
 ['DEMO', 'DEMO', 'DEMO', 'DEMO'],
 ['DEMO', 'DEMO', 'DEMO', 'DEMO'],
 ['DEMO', 'DEMO', 'DEMO', 'DEMO'],
 ['DEMO', 'DEMO', 'DEMO', 'DEMO'],
 ['DEMO', 'DEMO', 'DEMO', 'DEMO'],
 ['DEMO', 'DEMO', 'DEMO', 'DEMO'],
 ['DEMO', 'DEMO', 'DEMO', 'DEMO', 'ATC', 'ATC'],
 ['DEMO',
  'DEMO',
  'DEMO',
  'DEMO',
  'LABS',
  'LABS',
  'LABS',
  'LABS',
  'LABS',
  'LABS',
  'LABS',
  'LABS',
  'LABS',
  'LABS',
  'LABS'

In [21]:
table['VECTOR'][250417]

[['Ambulatório', '48.0', 'Ortopedia', 'M'],
 ['Ambulatório', '48.0', 'Radiologia', 'M'],
 ['Ambulatório', '48.0', 'Dermatologia', 'M'],
 ['Ambulatório',
  '48.0',
  'Patologia Clínica',
  'M',
  '4544-3',
  '2951-2',
  '718-7',
  '789-8',
  '777-3',
  '785-6',
  '1742-6',
  '11572-5',
  '1920-8',
  '6690-2',
  '787-2',
  '788-0',
  '786-4',
  '4537-7',
  '713-8',
  '14933-6',
  '5905-5',
  '13929-5',
  '736-9',
  '22664-7',
  '2324-2',
  '2823-3',
  '770-8',
  '14682-9',
  '706-2'],
 ['Ambulatório', '48.0', 'Ortopedia', 'M'],
 ['Urgência', '49.0', 'AMP', 'Radiologia', 'M', 'N02BE01', 'A03FA01'],
 ['Ambulatório', '49.0', 'Cardiologia', 'M'],
 ['Ambulatório', '50.0', 'Radiologia', 'M'],
 ['Ambulatório', '53.0', 'Radiologia', 'M'],
 ['Ambulatório', '56.0', 'Oftalmologia', 'M'],
 ['Ambulatório', '57.0', 'Angiologia e Cirurgia Vascular', 'M'],
 ['Ambulatório', '57.0', 'Medicina Interna', 'M'],
 ['Ambulatório', '58.0', 'Angiologia e Cirurgia Vascular', 'M'],
 ['Ambulatório', '58.0', 'Medicin

In [22]:
# FLatten list (takes around 3 min)
table['VECTOR'] = table['VECTOR'].map(lambda s: [item for sublist in s for item in sublist])

In [23]:
# Flatten list
table['INFO'] = table['INFO'].map(lambda z: [item for sublist in z for item in sublist])

In [24]:
table['VECTOR']

0         [Ambulatório, 73.0, Ortopedia, F, Ambulatório,...
1         [Ambulatório, 62.0, Medicina Interna, Radiolog...
2         [Ambulatório, 59.0, Cardiologia, M, Ambulatóri...
3         [Ambulatório, 64.0, Radiologia, F, Ambulatório...
4         [Ambulatório, 46.0, Ginecologia/Obstetrícia, F...
                                ...                        
250413    [Ambulatório, 78.0, Cardiologia, F, C07AB07, N...
250414    [Ambulatório, 61.0, Dermatologia, M, D11AH01, ...
250415    [Ambulatório, 55.0, Endocrinologia, M, A10BA02...
250416    [Ambulatório, 62.0, Medicina Geral e Familiar,...
250417    [Ambulatório, 48.0, Ortopedia, M, Ambulatório,...
Name: VECTOR, Length: 250418, dtype: object

In [25]:
table['VECTOR'][250417]

['Ambulatório',
 '48.0',
 'Ortopedia',
 'M',
 'Ambulatório',
 '48.0',
 'Radiologia',
 'M',
 'Ambulatório',
 '48.0',
 'Dermatologia',
 'M',
 'Ambulatório',
 '48.0',
 'Patologia Clínica',
 'M',
 '4544-3',
 '2951-2',
 '718-7',
 '789-8',
 '777-3',
 '785-6',
 '1742-6',
 '11572-5',
 '1920-8',
 '6690-2',
 '787-2',
 '788-0',
 '786-4',
 '4537-7',
 '713-8',
 '14933-6',
 '5905-5',
 '13929-5',
 '736-9',
 '22664-7',
 '2324-2',
 '2823-3',
 '770-8',
 '14682-9',
 '706-2',
 'Ambulatório',
 '48.0',
 'Ortopedia',
 'M',
 'Urgência',
 '49.0',
 'AMP',
 'Radiologia',
 'M',
 'N02BE01',
 'A03FA01',
 'Ambulatório',
 '49.0',
 'Cardiologia',
 'M',
 'Ambulatório',
 '50.0',
 'Radiologia',
 'M',
 'Ambulatório',
 '53.0',
 'Radiologia',
 'M',
 'Ambulatório',
 '56.0',
 'Oftalmologia',
 'M',
 'Ambulatório',
 '57.0',
 'Angiologia e Cirurgia Vascular',
 'M',
 'Ambulatório',
 '57.0',
 'Medicina Interna',
 'M',
 'Ambulatório',
 '58.0',
 'Angiologia e Cirurgia Vascular',
 'M',
 'Ambulatório',
 '58.0',
 'Medicina Interna',
 '

In [26]:
table['INFO'][250417]

['DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'LABS',
 'LABS',
 'LABS',
 'LABS',
 'LABS',
 'LABS',
 'LABS',
 'LABS',
 'LABS',
 'LABS',
 'LABS',
 'LABS',
 'LABS',
 'LABS',
 'LABS',
 'LABS',
 'LABS',
 'LABS',
 'LABS',
 'LABS',
 'LABS',
 'LABS',
 'LABS',
 'LABS',
 'LABS',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'ATC',
 'ATC',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'ATC',
 'ATC',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'LABS',
 'LABS',
 'LABS',
 'LABS',
 'LABS',
 'LABS',
 'LABS',
 'LABS',
 'LABS',
 'LABS',
 'LABS',
 'LABS',
 'LABS',
 'LAB

In [27]:
def is_date_string(s):
    # return re.search(r'^\d{4}-\d{2}-\d{2}$')
    return '-' in s and s[:4].isdigit()

In [28]:
# Remove dates
# table['VECTOR'] = table['VECTOR'].map(lambda x: [s for s in x if not is_date_string(s)])

In [30]:
# Remove dates
# table['INFO'] = table['INFO'].map(lambda x: [s for s in x if not is_date_string(s)])

In [31]:
# table['VECTOR'][250417]

In [32]:
# table['INFO'][250417]

In [33]:
listed_info = table['INFO']

In [34]:
listed_vector = table['VECTOR']

In [35]:
listed_vector

0         [Ambulatório, 73.0, Ortopedia, F, Ambulatório,...
1         [Ambulatório, 62.0, Medicina Interna, Radiolog...
2         [Ambulatório, 59.0, Cardiologia, M, Ambulatóri...
3         [Ambulatório, 64.0, Radiologia, F, Ambulatório...
4         [Ambulatório, 46.0, Ginecologia/Obstetrícia, F...
                                ...                        
250413    [Ambulatório, 78.0, Cardiologia, F, C07AB07, N...
250414    [Ambulatório, 61.0, Dermatologia, M, D11AH01, ...
250415    [Ambulatório, 55.0, Endocrinologia, M, A10BA02...
250416    [Ambulatório, 62.0, Medicina Geral e Familiar,...
250417    [Ambulatório, 48.0, Ortopedia, M, Ambulatório,...
Name: VECTOR, Length: 250418, dtype: object

In [36]:
listed_vector[0]

['Ambulatório',
 '73.0',
 'Ortopedia',
 'F',
 'Ambulatório',
 '74.0',
 'Gastrenterologia',
 'Radiologia',
 'F',
 'Ambulatório',
 '74.0',
 'Oftalmologia',
 'F',
 'Ambulatório',
 '76.0',
 'Gastrenterologia',
 'F',
 'Ambulatório',
 '77.0',
 'Cardiologia',
 'F',
 'Ambulatório',
 '77.0',
 'Cardiologia',
 'F',
 'C08CA13',
 'C09DA06',
 'Ambulatório',
 '77.0',
 'Ortopedia',
 'F',
 'A02BC01',
 'M01AB05',
 'Ambulatório',
 '77.0',
 'Radiologia',
 'F',
 'Ambulatório',
 '77.0',
 'Ortopedia',
 'F',
 'M01AB05',
 'Ambulatório',
 '77.0',
 'Cardiologia',
 'F',
 'Ambulatório',
 '77.0',
 'Angiologia e Cirurgia Vascular',
 'F',
 'Ambulatório',
 '78.0',
 'Cardiologia',
 'F',
 'Ambulatório',
 '78.0',
 'Radiologia',
 'F',
 'Ambulatório',
 '78.0',
 'Ortopedia',
 'F',
 'Ambulatório',
 '78.0',
 'Radiologia',
 'F',
 'Ambulatório',
 '78.0',
 'Ortopedia',
 'F',
 'Ambulatório',
 '78.0',
 'Cardiologia',
 'F',
 'Ambulatório',
 '79.0',
 'Radiologia',
 'F',
 'Ambulatório',
 '79.0',
 'Radiologia',
 'F',
 'Ambulatório',
 

### One-hot encoding

In [37]:
# table['VECTOR'] = table['VECTOR'].map(lambda s: re.sub(r'\b0+(?!\b)', '', s))

In [38]:
# listed_vector = table['VECTOR']
listed_vector

0         [Ambulatório, 73.0, Ortopedia, F, Ambulatório,...
1         [Ambulatório, 62.0, Medicina Interna, Radiolog...
2         [Ambulatório, 59.0, Cardiologia, M, Ambulatóri...
3         [Ambulatório, 64.0, Radiologia, F, Ambulatório...
4         [Ambulatório, 46.0, Ginecologia/Obstetrícia, F...
                                ...                        
250413    [Ambulatório, 78.0, Cardiologia, F, C07AB07, N...
250414    [Ambulatório, 61.0, Dermatologia, M, D11AH01, ...
250415    [Ambulatório, 55.0, Endocrinologia, M, A10BA02...
250416    [Ambulatório, 62.0, Medicina Geral e Familiar,...
250417    [Ambulatório, 48.0, Ortopedia, M, Ambulatório,...
Name: VECTOR, Length: 250418, dtype: object

In [39]:
len(listed_vector)

250418

In [40]:
listed_vector[0]

['Ambulatório',
 '73.0',
 'Ortopedia',
 'F',
 'Ambulatório',
 '74.0',
 'Gastrenterologia',
 'Radiologia',
 'F',
 'Ambulatório',
 '74.0',
 'Oftalmologia',
 'F',
 'Ambulatório',
 '76.0',
 'Gastrenterologia',
 'F',
 'Ambulatório',
 '77.0',
 'Cardiologia',
 'F',
 'Ambulatório',
 '77.0',
 'Cardiologia',
 'F',
 'C08CA13',
 'C09DA06',
 'Ambulatório',
 '77.0',
 'Ortopedia',
 'F',
 'A02BC01',
 'M01AB05',
 'Ambulatório',
 '77.0',
 'Radiologia',
 'F',
 'Ambulatório',
 '77.0',
 'Ortopedia',
 'F',
 'M01AB05',
 'Ambulatório',
 '77.0',
 'Cardiologia',
 'F',
 'Ambulatório',
 '77.0',
 'Angiologia e Cirurgia Vascular',
 'F',
 'Ambulatório',
 '78.0',
 'Cardiologia',
 'F',
 'Ambulatório',
 '78.0',
 'Radiologia',
 'F',
 'Ambulatório',
 '78.0',
 'Ortopedia',
 'F',
 'Ambulatório',
 '78.0',
 'Radiologia',
 'F',
 'Ambulatório',
 '78.0',
 'Ortopedia',
 'F',
 'Ambulatório',
 '78.0',
 'Cardiologia',
 'F',
 'Ambulatório',
 '79.0',
 'Radiologia',
 'F',
 'Ambulatório',
 '79.0',
 'Radiologia',
 'F',
 'Ambulatório',
 

In [41]:
# listed_info = table['INFO']
listed_info

0         [DEMO, DEMO, DEMO, DEMO, DEMO, DEMO, DEMO, DEM...
1         [DEMO, DEMO, DEMO, DEMO, DEMO, DEMO, DEMO, DEM...
2         [DEMO, DEMO, DEMO, DEMO, DEMO, DEMO, DEMO, DEM...
3         [DEMO, DEMO, DEMO, DEMO, DEMO, DEMO, DEMO, DEM...
4         [DEMO, DEMO, DEMO, DEMO, DEMO, DEMO, DEMO, DEM...
                                ...                        
250413    [DEMO, DEMO, DEMO, DEMO, ATC, ATC, ATC, DEMO, ...
250414    [DEMO, DEMO, DEMO, DEMO, ATC, DEMO, DEMO, DEMO...
250415    [DEMO, DEMO, DEMO, DEMO, ATC, DEMO, DEMO, DEMO...
250416    [DEMO, DEMO, DEMO, DEMO, DEMO, DEMO, DEMO, DEM...
250417    [DEMO, DEMO, DEMO, DEMO, DEMO, DEMO, DEMO, DEM...
Name: INFO, Length: 250418, dtype: object

In [42]:
listed_info[0]

['DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'ATC',
 'ATC',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'ATC',
 'ATC',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'ATC',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'ATC',
 'ATC',
 'ATC',
 'ATC',
 'ATC',
 'ATC',
 'ATC',
 'ATC',
 'ATC',
 'ATC',
 'ATC',
 'ATC',
 'ATC',
 'ATC',
 'ATC',
 'ATC',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'DEMO',
 'ICD9',
 'IC

In [43]:
# Map ICD9 codes to CCS example
icd9_3 = icdmap.lookup('icd9_level3', '250.10')
icd9_3

'250'

In [44]:
ccs = icdmap.lookup('icd9_3toccs', icd9_3)
ccs

50

In [45]:
l_info1 = [['DEMO', 'DEMO', 'DEMO', 'DEMO', 'ICD9', 'CRON', 'CRON', 'CRON', 'CRON', 'CRON', 'PAT', 'PAT', 'PAT', 'PAT'], ['DEMO', 'DEMO', 'DEMO', 'DEMO']]
l_info1 = listed_info
l_info1

0         [DEMO, DEMO, DEMO, DEMO, DEMO, DEMO, DEMO, DEM...
1         [DEMO, DEMO, DEMO, DEMO, DEMO, DEMO, DEMO, DEM...
2         [DEMO, DEMO, DEMO, DEMO, DEMO, DEMO, DEMO, DEM...
3         [DEMO, DEMO, DEMO, DEMO, DEMO, DEMO, DEMO, DEM...
4         [DEMO, DEMO, DEMO, DEMO, DEMO, DEMO, DEMO, DEM...
                                ...                        
250413    [DEMO, DEMO, DEMO, DEMO, ATC, ATC, ATC, DEMO, ...
250414    [DEMO, DEMO, DEMO, DEMO, ATC, DEMO, DEMO, DEMO...
250415    [DEMO, DEMO, DEMO, DEMO, ATC, DEMO, DEMO, DEMO...
250416    [DEMO, DEMO, DEMO, DEMO, DEMO, DEMO, DEMO, DEM...
250417    [DEMO, DEMO, DEMO, DEMO, DEMO, DEMO, DEMO, DEM...
Name: INFO, Length: 250418, dtype: object

In [46]:
"""
res_li1 = []

for item in listed_info: 
    res_li1.append(item)
"""

'\nres_li1 = []\n\nfor item in listed_info: \n    res_li1.append(item)\n'

In [47]:
# res_li1

In [48]:
l_vector1 = [['Ambulatório', '59.0', 'Cirurgia Plástica', 'Reconstrutiva e Estética', 'F', '174', 'Obesity', 'Depression', 'HTN', 'Breast malignancy', 'Thyroid disorders', 'Endocrine', 'Psychiatric', 'Cardiovascular', 'Oncology'], ['Ambulatório', '59.0', 'Medicina Física e Reabilitação', 'F']]
l_vector1 = listed_vector
l_vector1

0         [Ambulatório, 73.0, Ortopedia, F, Ambulatório,...
1         [Ambulatório, 62.0, Medicina Interna, Radiolog...
2         [Ambulatório, 59.0, Cardiologia, M, Ambulatóri...
3         [Ambulatório, 64.0, Radiologia, F, Ambulatório...
4         [Ambulatório, 46.0, Ginecologia/Obstetrícia, F...
                                ...                        
250413    [Ambulatório, 78.0, Cardiologia, F, C07AB07, N...
250414    [Ambulatório, 61.0, Dermatologia, M, D11AH01, ...
250415    [Ambulatório, 55.0, Endocrinologia, M, A10BA02...
250416    [Ambulatório, 62.0, Medicina Geral e Familiar,...
250417    [Ambulatório, 48.0, Ortopedia, M, Ambulatório,...
Name: VECTOR, Length: 250418, dtype: object

In [49]:
nhc = table['NHC']
d_h_a = table['prediction_time']

In [50]:
nhc1 = ['OIOIOI', 'HIHIHI']
d_h_a1 = ['2360-01-11', '2360-02-11']

In [51]:
listed_vector

0         [Ambulatório, 73.0, Ortopedia, F, Ambulatório,...
1         [Ambulatório, 62.0, Medicina Interna, Radiolog...
2         [Ambulatório, 59.0, Cardiologia, M, Ambulatóri...
3         [Ambulatório, 64.0, Radiologia, F, Ambulatório...
4         [Ambulatório, 46.0, Ginecologia/Obstetrícia, F...
                                ...                        
250413    [Ambulatório, 78.0, Cardiologia, F, C07AB07, N...
250414    [Ambulatório, 61.0, Dermatologia, M, D11AH01, ...
250415    [Ambulatório, 55.0, Endocrinologia, M, A10BA02...
250416    [Ambulatório, 62.0, Medicina Geral e Familiar,...
250417    [Ambulatório, 48.0, Ortopedia, M, Ambulatório,...
Name: VECTOR, Length: 250418, dtype: object

In [52]:
listed_info

0         [DEMO, DEMO, DEMO, DEMO, DEMO, DEMO, DEMO, DEM...
1         [DEMO, DEMO, DEMO, DEMO, DEMO, DEMO, DEMO, DEM...
2         [DEMO, DEMO, DEMO, DEMO, DEMO, DEMO, DEMO, DEM...
3         [DEMO, DEMO, DEMO, DEMO, DEMO, DEMO, DEMO, DEM...
4         [DEMO, DEMO, DEMO, DEMO, DEMO, DEMO, DEMO, DEM...
                                ...                        
250413    [DEMO, DEMO, DEMO, DEMO, ATC, ATC, ATC, DEMO, ...
250414    [DEMO, DEMO, DEMO, DEMO, ATC, DEMO, DEMO, DEMO...
250415    [DEMO, DEMO, DEMO, DEMO, ATC, DEMO, DEMO, DEMO...
250416    [DEMO, DEMO, DEMO, DEMO, DEMO, DEMO, DEMO, DEM...
250417    [DEMO, DEMO, DEMO, DEMO, DEMO, DEMO, DEMO, DEM...
Name: INFO, Length: 250418, dtype: object

In [53]:
repeated = ['LABS', 'CRON', 'PAT', 'ATC', 'ICD9', 'DEMO', 'MCDTS']

def construct_rows_list2(listed_vector, listed_info, nhc, d_h_a):
    """
    Converts into dictionary format, inserting 1's for the one-hot encoding. 
    """
    # it = 0
    rows_list = []

    for vector, info, nhc_id, time_stamp in tqdm(zip(listed_vector, listed_info, nhc, d_h_a), total=len(listed_vector)):

        dict1 = {} # Dict with info for the event with the whole columns
        add = True
        dict1['NHC'] = nhc_id
        dict1['DATA_HORA_ADMISSAO'] = time_stamp
        aux = 0

        for idx, (v, i) in enumerate(zip(vector, info)):
            
            # FIX: special cases where ICD9 referes incorrectly to a specialty or gender information 
            if i == 'ICD9' and (len(v.split()) > 1 or v == 'F' or v == 'M' or v.isalpha()):
                # print(it)
                # print(nhc_id)
                # print(v)
                add = False

            if i in repeated:
                if i == 'DEMO' and add:
                    if aux == 0:
                        dict1['TIPO_EP'] = v
                    if aux == 1:
                        dict1['IDADE'] = v
                    if aux == 2:
                        dict1['ESPECIALIDADE'] = v
                        if vector[idx+1] != 'F' and vector[idx+1] != 'M':
                            # Next case: aux = 2, so re-enters specialty
                            # Special cases where more than one specialty
                            aux = 1
                    if aux == 3:
                        dict1['GENERO'] = v
                    
                    aux += 1

                elif  i == 'ICD9' and add:
                    # Convert ICD9 to CCS
                    icd9_3 = icdmap.lookup('icd9_level3', v)
                    ccs = icdmap.lookup('icd9_3toccs', icd9_3)
                    dict1[i+'_'+str(ccs)] = 1
                else:
                    dict1[i+'_'+str(v)] = 1
            else:
                dict1[i] = v
        
        if add:
            rows_list.append(dict1)

        # it += 1
            
    return rows_list

In [54]:
rows_list = construct_rows_list2(listed_vector, listed_info, nhc, d_h_a)
len(rows_list)

100%|██████████| 250418/250418 [01:47<00:00, 2326.07it/s]


175398

In [56]:
# FIX: Examples -> 'LABS_Ambulatório': 1, 'LABS_62.0': 1,  'LABS_Cardiologia': 1,
rows_list[0]

{'NHC': '01412FD488D32C591826DEC62C02B178',
 'DATA_HORA_ADMISSAO': '2340-07-08',
 'TIPO_EP': 'Ambulatório',
 'IDADE': '73.0',
 'ESPECIALIDADE': 'Ortopedia',
 'GENERO': 'F',
 'ATC_C08CA13': 1,
 'ATC_C09DA06': 1,
 'ATC_Radiologia': 1,
 'ATC_F': 1,
 'ATC_M01AB05': 1,
 'ATC_C10AA05': 1,
 'ATC_B05BB01': 1,
 'ATC_C07AB07': 1,
 'ATC_A04AA01': 1,
 'ATC_A10AB04': 1,
 'ATC_N02AX02': 1,
 'ATC_B05BB02': 1,
 'ATC_N02BE01': 1,
 'ATC_N02AB03': 1,
 'ATC_N05BA08': 1,
 'ATC_A02BC02': 1,
 'ATC_M01AB15': 1,
 'ATC_J01DB04': 1,
 'ATC_M04AA01': 1,
 'ATC_N02BB02': 1,
 'ICD9_238': 1,
 'ICD9_50': 1,
 'ICD9_83': 1,
 'ICD9_99': 1,
 'ICD9_157': 1,
 'LABS_1742-6': 1,
 'LABS_2951-2': 1,
 'LABS_3243-3': 1,
 'LABS_785-6': 1,
 'LABS_6768-6': 1,
 'LABS_14631-6': 1,
 'LABS_20453-7': 1,
 'LABS_777-3': 1,
 'LABS_770-8': 1,
 'LABS_20577-3': 1,
 'LABS_2823-3': 1,
 'LABS_29576-6': 1,
 'LABS_15152-2': 1,
 'LABS_2324-2': 1,
 'LABS_789-8': 1,
 'LABS_786-4': 1,
 'LABS_713-8': 1,
 'LABS_13315-7': 1,
 'LABS_1988-5': 1,
 'LABS_14879

In [57]:
# FIX: why null value?
# Category of features
s = set()

for row in listed_info:
    # print(row)
    s.update(row)
s

{'ATC', 'CRON', 'DEMO', 'ICD9', 'LABS', 'MCDTS', 'PAT'}

In [58]:
possible_values = {k:[] for k in list(s)}

for vector, info in tqdm(zip(listed_vector,listed_info), total=len(listed_vector)):
    """
    Possible values for each category. 
    """
    add = True
    for v, i in zip(vector, info):
        if i == 'ICD9' and len(v.split()) > 1:
            # print(v)
            add = False
        # FIX: wrong values (specialty, gender, episode type) of vector being coded as ICD9
        if add and i == 'ICD9' and add and v != 'F' and v != 'M' and not v.isalpha():
            # Convert ICD9 to CCS
            icd9_3 = icdmap.lookup('icd9_level3',str(v))
            ccs = icdmap.lookup('icd9_3toccs', icd9_3)
            if ccs not in possible_values[i]:
                # print(ccs)
                possible_values[i].append(ccs)
            
        if v not in possible_values[i] and add and i != 'ICD9':
            possible_values[i].append(v)

100%|██████████| 250418/250418 [03:55<00:00, 1061.49it/s]


In [59]:
possible_values['ICD9']

[238,
 50,
 83,
 99,
 157,
 108,
 48,
 158,
 163,
 205,
 653,
 164,
 203,
 58,
 200,
 113,
 91,
 253,
 nan,
 173,
 239,
 152,
 670,
 134,
 136,
 15,
 86,
 59,
 226,
 244,
 159,
 250,
 126,
 7,
 211,
 225,
 175,
 232,
 90,
 147,
 95,
 259,
 237,
 88,
 149,
 659,
 249,
 197,
 657,
 96,
 14,
 2,
 107,
 122,
 115,
 251,
 94,
 204,
 166,
 127,
 217,
 100,
 101,
 97,
 660,
 143,
 120,
 24,
 231,
 198,
 6,
 54,
 25,
 47,
 121,
 44,
 11,
 212,
 162,
 8,
 146,
 109,
 153,
 165,
 117,
 202,
 28,
 39,
 224,
 89,
 112,
 128,
 658,
 655,
 105,
 93,
 51,
 195,
 52,
 156,
 167,
 248,
 235,
 160,
 29,
 13,
 18,
 17,
 61,
 78,
 247,
 118,
 35,
 119,
 209,
 36,
 145,
 144,
 135,
 130,
 230,
 42,
 125,
 84,
 132,
 169,
 87,
 229,
 4,
 124,
 110,
 663,
 32,
 141,
 20,
 40,
 170,
 116,
 1,
 23,
 174,
 41,
 38,
 3,
 123,
 236,
 34,
 92,
 16,
 46,
 242,
 151,
 133,
 181,
 199,
 180,
 184,
 208,
 63,
 168,
 193,
 142,
 22,
 213,
 137,
 103,
 80,
 240,
 161,
 111,
 81,
 37,
 214,
 215,
 114,
 183,
 201,
 234,


In [60]:
# Construction of columns vector
columns = []

for key in tqdm(possible_values.keys(), total=len(possible_values.keys())):
    """
    Construction of columns vector.
    """
    if key not in repeated:
        print(key)
        columns.append(key)
    else:
        for c in possible_values[key]:
            columns.append(key+'_'+str(c))
            # print(key+'_'+str(c))

print('Number of columns: ', len(columns)) #size vector

100%|██████████| 7/7 [00:00<00:00, 1740.79it/s]

Number of columns:  8002


In [61]:
columns

['ATC_C08CA13',
 'ATC_C09DA06',
 'ATC_Radiologia',
 'ATC_F',
 'ATC_M01AB05',
 'ATC_C10AA05',
 'ATC_B05BB01',
 'ATC_C07AB07',
 'ATC_A04AA01',
 'ATC_A10AB04',
 'ATC_N02AX02',
 'ATC_B05BB02',
 'ATC_N02BE01',
 'ATC_N02AB03',
 'ATC_N05BA08',
 'ATC_A02BC02',
 'ATC_M01AB15',
 'ATC_J01DB04',
 'ATC_M04AA01',
 'ATC_N02BB02',
 'ATC_B01AB01',
 'ATC_B01AB05',
 'ATC_J01MA02',
 'ATC_B05XA03',
 'ATC_B05BA03',
 'ATC_C08CA01',
 'ATC_A06AD11',
 'ATC_N03AX14',
 'ATC_C10AA07',
 'ATC_TCCE',
 'ATC_Urgência',
 'ATC_83.0',
 'ATC_AMP',
 'ATC_C01EB10',
 'ATC_N05AH04',
 'ATC_A12BA01',
 'ATC_H03AA01',
 'ATC_A12CC05',
 'ATC_B05XA01',
 'ATC_C09CA03',
 'ATC_C03CA01',
 'ATC_S01AA11',
 'ATC_H02AB09',
 'ATC_A10AB01',
 'ATC_R03AC02',
 'ATC_C03DA01',
 'ATC_C01CA03',
 'ATC_A10AE04',
 'ATC_A06AB02',
 'ATC_B05XA05',
 'ATC_R03BB01',
 'ATC_786-4',
 'ATC_4544-3',
 'ATC_5767-9',
 'ATC_20405-7',
 'ATC_2823-3',
 'ATC_14928-6',
 'ATC_785-6',
 'ATC_5794-3',
 'ATC_15042-5',
 'ATC_5792-7',
 'ATC_11556-8',
 'ATC_5778-6',
 'ATC_A10BH02'

In [62]:
df = []
for row in tqdm(rows_list, total=len(rows_list)):   
    init_dict = {c:0 for c in columns}
    init_dict.update(row)
    df.append(init_dict)

100%|██████████| 175398/175398 [02:22<00:00, 1235.13it/s]


In [63]:
df[0]

{'ATC_C08CA13': 1,
 'ATC_C09DA06': 1,
 'ATC_Radiologia': 1,
 'ATC_F': 1,
 'ATC_M01AB05': 1,
 'ATC_C10AA05': 1,
 'ATC_B05BB01': 1,
 'ATC_C07AB07': 1,
 'ATC_A04AA01': 1,
 'ATC_A10AB04': 1,
 'ATC_N02AX02': 1,
 'ATC_B05BB02': 1,
 'ATC_N02BE01': 1,
 'ATC_N02AB03': 1,
 'ATC_N05BA08': 1,
 'ATC_A02BC02': 1,
 'ATC_M01AB15': 1,
 'ATC_J01DB04': 1,
 'ATC_M04AA01': 1,
 'ATC_N02BB02': 1,
 'ATC_B01AB01': 1,
 'ATC_B01AB05': 1,
 'ATC_J01MA02': 1,
 'ATC_B05XA03': 1,
 'ATC_B05BA03': 1,
 'ATC_C08CA01': 1,
 'ATC_A06AD11': 1,
 'ATC_N03AX14': 1,
 'ATC_C10AA07': 1,
 'ATC_TCCE': 1,
 'ATC_Urgência': 1,
 'ATC_83.0': 1,
 'ATC_AMP': 1,
 'ATC_C01EB10': 1,
 'ATC_N05AH04': 1,
 'ATC_A12BA01': 1,
 'ATC_H03AA01': 1,
 'ATC_A12CC05': 1,
 'ATC_B05XA01': 1,
 'ATC_C09CA03': 1,
 'ATC_C03CA01': 1,
 'ATC_S01AA11': 1,
 'ATC_H02AB09': 1,
 'ATC_A10AB01': 1,
 'ATC_R03AC02': 1,
 'ATC_C03DA01': 1,
 'ATC_C01CA03': 1,
 'ATC_A10AE04': 1,
 'ATC_A06AB02': 1,
 'ATC_B05XA05': 1,
 'ATC_R03BB01': 1,
 'ATC_786-4': 1,
 'ATC_4544-3': 1,
 'ATC_57

In [64]:
table = pd.DataFrame(df) # This will take some time :)
table.head()

In [ ]:
# Save this as it took a long time, and read again from here if the notebook goes down
table.to_csv('diabetes_init_df7.csv')

In [ ]:
del df

In [ ]:
len(table)

245396

In [ ]:
# Number of events per NHC
vc = table['NHC'].value_counts()
vc

01412FD488D32C591826DEC62C02B178    62
E7107B9C839B51050DB4DA11428A3DCF    62
ECC34C3FBFDC408DBCA51C8F54BAC537    62
ED83A66B85FD4505F4470F9C7DD24E64    62
EEF5B6A10B7D85A800FABF71AB6CB034    62
                                    ..
A819573B252BBD3A7AC0FDDD4951A54D    62
A825B03188C75CD9403462C865130D9E    62
AA2C632FB376B5288D0325594D3E515A    62
AA9D4F822E617D7AE40D3AE0A83668D1    62
FEB96E6637ACCF6B46A683F4C68A5EC5    62
Name: NHC, Length: 3958, dtype: int64

In [ ]:
table.head()

,null,MCDTS_TCABD,MCDTS_TCCE,MCDTS_TCPELV,MCDTS_TCTX,MCDTS_TCANGIOTRX,MCDTS_ECOABD,MCDTS_ECOVES,MCDTS_ECORENAL,MCDTS_TCCONTORAL,...,ATC_2639-3,ATC_13457-7,ATC_L04AA13,ATC_A06AC51,ATC_D10AX03,ATC_A10BD10,NHC,DATA_HORA_ADMISSAO,ICD9_F,ICD9_M
0,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,01412FD488D32C591826DEC62C02B178,2340-07-08,NaN,NaN
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,050E246D80AA2F1BACD35F4C06CB1525,2340-07-08,NaN,NaN
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,054760511AEF49C14D9F2A98A70C98C5,2340-07-08,NaN,NaN
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,08981B30B3CE45D82CE414AF73A3F48F,2340-07-08,NaN,NaN
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0C564E336B0DCB433EA6C9D072456383,2340-07-08,NaN,NaN


In [ ]:
vc.to_csv('vc_diabetes5.csv')

In [ ]:
table[table.NHC == '054760511AEF49C14D9F2A98A70C98C5']

,null,MCDTS_TCABD,MCDTS_TCCE,MCDTS_TCPELV,MCDTS_TCTX,MCDTS_TCANGIOTRX,MCDTS_ECOABD,MCDTS_ECOVES,MCDTS_ECORENAL,MCDTS_TCCONTORAL,...,ATC_2639-3,ATC_13457-7,ATC_L04AA13,ATC_A06AC51,ATC_D10AX03,ATC_A10BD10,NHC,DATA_HORA_ADMISSAO,ICD9_F,ICD9_M
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,054760511AEF49C14D9F2A98A70C98C5,2340-07-08,NaN,NaN
3960,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,054760511AEF49C14D9F2A98A70C98C5,2341-01-02,NaN,NaN
7918,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,054760511AEF49C14D9F2A98A70C98C5,2341-06-29,NaN,NaN
11876,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,054760511AEF49C14D9F2A98A70C98C5,2341-12-24,NaN,NaN
15834,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,054760511AEF49C14D9F2A98A70C98C5,2342-06-20,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225608,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,054760511AEF49C14D9F2A98A70C98C5,2368-04-18,NaN,NaN
229566,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,054760511AEF49C14D9F2A98A70C98C5,2368-10-13,NaN,NaN
233524,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,054760511AEF49C14D9F2A98A70C98C5,2369-04-09,NaN,NaN
237482,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,054760511AEF49C14D9F2A98A70C98C5,2369-10-04,NaN,NaN
